# TalkingData AdTracking Fraud Detection Challenge

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
import numpy as np
np.set_printoptions(threshold=50, edgeitems=20)

PATH='data/talkingdata/'

In [ ]:
# !mkdir $PATH
# !kaggle competitions download -c talkingdata-adtracking-fraud-detection -p ~/github/fastai/courses/dl1/$PATH

In [ ]:
# !unzip -q -o '{PATH}*.zip' -d {PATH}

In [ ]:
# !rm -r {PATH}*.zip

## Datasets

In [3]:
train_df = pd.read_csv(f'{PATH}train_sample.csv')
train_df["click_time"] = pd.to_datetime(train_df.click_time)
# test_df = pd.read_csv(f'{PATH}test.csv')
# test_df["click_time"] = pd.to_datetime(test_df.click_time)

In [ ]:
train_df.head()
# test_df.head()

In [ ]:
dates = train_df["click_time"]
plt.hist(dates, bins=100)

In [4]:
add_datepart(train_df, "click_time", drop=False, time=True)
# add_datepart(test_df, "click_time", drop=False, time=True)

In [ ]:
train_df.head(10)

In [5]:
train_df.drop(['attributed_time','click_timeYear','click_timeMonth','click_timeIs_month_end','click_timeIs_month_start'
             ,'click_timeIs_quarter_end','click_timeIs_quarter_start','click_timeIs_year_end','click_timeIs_year_end'
             , 'click_timeIs_year_start', 'click_timeElapsed'], axis=1, inplace=True)
# test_df = train_df[cols]

In [6]:
add_datepart(train_df, "click_time", drop=False)
# add_datepart(test_df, "click_time", drop=False)

In [7]:
train_df.head(10)

,ip,app,device,os,channel,click_time,is_attributed,click_timeWeek,click_timeDay,click_timeDayofweek,...,click_timeSecond,click_timeYear,click_timeMonth,click_timeIs_month_end,click_timeIs_month_start,click_timeIs_quarter_end,click_timeIs_quarter_start,click_timeIs_year_end,click_timeIs_year_start,click_timeElapsed
0,87540,12,1,13,497,2017-11-07 09:30:38,0,45,7,1,...,38,2017,11,False,False,False,False,False,False,1510047038
1,105560,25,1,17,259,2017-11-07 13:40:27,0,45,7,1,...,27,2017,11,False,False,False,False,False,False,1510062027
2,101424,12,1,19,212,2017-11-07 18:05:24,0,45,7,1,...,24,2017,11,False,False,False,False,False,False,1510077924
3,94584,13,1,13,477,2017-11-07 04:58:08,0,45,7,1,...,8,2017,11,False,False,False,False,False,False,1510030688
4,68413,12,1,1,178,2017-11-09 09:00:09,0,45,9,3,...,9,2017,11,False,False,False,False,False,False,1510218009
5,93663,3,1,17,115,2017-11-09 01:22:13,0,45,9,3,...,13,2017,11,False,False,False,False,False,False,1510190533
6,17059,1,1,17,135,2017-11-09 01:17:58,0,45,9,3,...,58,2017,11,False,False,False,False,False,False,1510190278
7,121505,9,1,25,442,2017-11-07 10:01:53,0,45,7,1,...,53,2017,11,False,False,False,False,False,False,1510048913
8,192967,2,2,22,364,2017-11-08 09:35:17,0,45,8,2,...,17,2017,11,False,False,False,False,False,False,1510133717
9,143636,3,1,19,135,2017-11-08 12:35:26,0,45,8,2,...,26,2017,11,False,False,False,False,False,False,1510144526


In [8]:
train_df.drop(['click_timeWeek','click_timeDay','click_timeYear','click_timeYear', 'click_timeDayofyear'
             ,'click_timeMonth','click_timeIs_month_end','click_timeIs_month_start','click_timeIs_quarter_end'
             , 'click_timeIs_quarter_start', 'click_timeIs_year_end', 'click_timeIs_year_start', 'click_timeElapsed']
              , axis=1, inplace=True)

In [9]:
train_df.head()
# test_df.head()

,ip,app,device,os,channel,click_time,is_attributed,click_timeDayofweek,click_timeHour,click_timeMinute,click_timeSecond
0,87540,12,1,13,497,2017-11-07 09:30:38,0,1,9,30,38
1,105560,25,1,17,259,2017-11-07 13:40:27,0,1,13,40,27
2,101424,12,1,19,212,2017-11-07 18:05:24,0,1,18,5,24
3,94584,13,1,13,477,2017-11-07 04:58:08,0,1,4,58,8
4,68413,12,1,1,178,2017-11-09 09:00:09,0,3,9,0,9


In [58]:
df = train_df.sort_values(['ip', 'click_time'])

In [59]:
df.head(5)

,ip,app,device,os,channel,click_time,is_attributed,click_timeDayofweek,click_timeHour,click_timeMinute,click_timeSecond
4249,9,9,1,13,244,2017-11-07 16:06:42,0,1,16,6,42
18992,10,11,1,22,319,2017-11-07 01:37:19,0,1,1,37,19
25399,10,12,1,19,140,2017-11-07 07:33:04,0,1,7,33,4
51193,10,18,1,13,107,2017-11-08 11:19:15,0,2,11,19,15
10686,19,14,1,16,379,2017-11-08 09:12:26,0,2,9,12,26


In [54]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'm')
    last_date = np.datetime64()
    last_ip = 0
    res = []

    for s,v,d in zip(df.ip.values, df[fld].values, df.click_time.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

In [56]:
fld='os'
get_elapsed(fld, 'hours_since_last_seen')

In [20]:
display(DataFrameSummary(df).summary())

,ip,app,device,os,channel,click_time,is_attributed,click_timeDayofweek,click_timeHour,click_timeMinute,click_timeSecond,Afterip
count,100000,100000,100000,100000,100000,NaN,100000,100000,100000,100000,100000,100000
mean,91255.9,12.0479,21.7712,22.8183,268.832,NaN,0.00227,1.86146,9.32859,29.1554,29.7799,0
std,69835.6,14.9415,259.668,55.9431,129.724,NaN,0.0475906,0.889266,6.18059,17.3697,17.2338,0
min,9,1,0,0,3,NaN,0,0,0,0,0,0
25%,40552,3,1,13,145,NaN,0,1,4,14,15,0
50%,79827,12,1,18,258,NaN,0,2,9,29,30,0
75%,118252,15,1,19,379,NaN,0,3,14,44,45,0
max,364757,551,3867,866,498,NaN,1,3,23,59,59,0
counts,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000,100000
uniques,34857,161,100,130,161,80350,2,4,24,60,60,1


In [ ]:
cat_vars = ['ip', 'app', 'device', 'os','channel', 'click_timeDayofweek', 'click_timeHour', 'click_timeMinute', 'click_timeSecond']

contin_vars = []

In [ ]:
dep = 'is_attributed'
train_df = train_df[cat_vars+contin_vars+[dep, 'click_time']].copy()
# test_df = test_df[cat_vars+contin_vars+[dep, 'click_time']].copy()

In [ ]:
for v in cat_vars: train_df[v] = train_df[v].astype('category').cat.as_ordered()

In [ ]:
apply_cats(test_df, train_df)

In [ ]:
train_df.reset_index(inplace=True)
# train_df = train_df.set_index("click_time")

In [ ]:
train_df['is_attributed'] = train_df['is_attributed'].astype('float32')

In [ ]:
display(DataFrameSummary(train_df).summary())

In [ ]:
train_df.to_feather(f'{PATH}train_df')
# test_df.to_feather(f'{PATH}test_df')

In [ ]:
train_df = pd.read_feather(f'{PATH}train_df')
# test_df.read_feather(f'{PATH}test_df')

In [ ]:
train_df = train_df.set_index("click_time")
train_df.head(2)

In [ ]:
train_df, y, _, mapper  = proc_df(train_df, 'is_attributed', do_scale=True)
y += 1
yl = np.log(y)

In [ ]:
val_idx = np.flatnonzero(
    (train_df.index<=datetime.datetime(2017 ,11 ,9 ,23 ,59)) & (train_df.index>=datetime.datetime(2017,11,9)))

In [ ]:
y

In [ ]:
val_idx

In [ ]:
# val_idx=[0]

## DL

We're ready to put together our models.

Root-mean-squared percent error is the metric Kaggle used for this competition.

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

We can create a ModelData object directly from out data frame.

In [ ]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, train_df, y.astype(np.float32), cat_flds=cat_vars, bs=128)

Some categorical variables have a lot more levels than others. Store, in particular, has over a thousand!

In [ ]:
cat_sz = [(c, len(train_df[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

We use the *cardinality* of each variable (that is, its number of unique values) to decide how large to make its *embeddings*. Each level will be associated with a vector with length defined as below.

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(emb_szs, len(train_df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot(100)

### All

In [ ]:
lr = 1e-3

In [ ]:
m.fit(lr, 1)

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)

### Test

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.save('val0')

In [ ]:
m.load('val0')

In [ ]:
x,y=m.predict_with_targs()

In [ ]:
exp_rmspe(x,y)

In [ ]:
pred_test=m.predict(True)

In [ ]:
pred_test = np.exp(pred_test)

In [ ]:
joined_test['Sales']=pred_test

In [ ]:
csv_fn=f'{PATH}tmp/sub.csv'

In [ ]:
joined_test[['Id','Sales']].to_csv(csv_fn, index=False)

In [ ]:
FileLink(csv_fn)